# Chapter 6 実践編2

## 6-4 文章を品詞分解

In [2]:
import MeCab

# 解析用のインスタンスを生成
mecab = MeCab.Tagger()
text = "私たちは人工知能を作るために勉強する"
# 解析結果を双方向リストの先頭ノードとして取得
node = mecab.parseToNode(text)

# "BOS/EOS,*..."は文頭, 文末を表す特殊なNode
while node:
    # surface: 単語, feature: 品詞
    print(f"{node.surface}\t{node.feature}")
    node = node.next

	BOS/EOS,*,*,*,*,*,*,*,*
私	名詞,代名詞,一般,*,*,*,私,ワタシ,ワタシ
たち	名詞,接尾,一般,*,*,*,たち,タチ,タチ
は	助詞,係助詞,*,*,*,*,は,ハ,ワ
人工	名詞,一般,*,*,*,*,人工,ジンコウ,ジンコー
知能	名詞,一般,*,*,*,*,知能,チノウ,チノー
を	助詞,格助詞,一般,*,*,*,を,ヲ,ヲ
作る	動詞,自立,*,*,五段・ラ行,基本形,作る,ツクル,ツクル
ため	名詞,非自立,副詞可能,*,*,*,ため,タメ,タメ
に	助詞,格助詞,一般,*,*,*,に,ニ,ニ
勉強	名詞,サ変接続,*,*,*,*,勉強,ベンキョウ,ベンキョー
する	動詞,自立,*,*,サ変・スル,基本形,する,スル,スル
	BOS/EOS,*,*,*,*,*,*,*,*


In [3]:
text = "にわにはにわにわとりがいる"
node = mecab.parseToNode(text)
# 文頭を表す特殊なノードを読み飛ばす
node = node.next

while node.next:
    print(f"{node.surface}\t{node.feature}")
    node = node.next

に	助詞,格助詞,一般,*,*,*,に,ニ,ニ
わに	名詞,一般,*,*,*,*,わに,ワニ,ワニ
はにわ	名詞,一般,*,*,*,*,はにわ,ハニワ,ハニワ
にわとり	名詞,一般,*,*,*,*,にわとり,ニワトリ,ニワトリ
が	助詞,格助詞,一般,*,*,*,が,ガ,ガ
いる	動詞,自立,*,*,一段,基本形,いる,イル,イル


In [4]:
# 注意：事前にdownload_rawdata.sh を実行して、3作家の文章をダウンロードしてください

from pathlib import Path
from collections import Counter
from typing import List
import re

DATASET_DIR = "./dataset/raw_datas"
STOPWORD_FILE = "./origin_stopwords.txt"
UNIQUE_FILE = "./origin_words.txt"
CORPUS_FILE = "./dataset/corpus.csv"

def main():
    # 分かち書きするオプションを指定してインスタンス生成
    mecab = MeCab.Tagger('-Owakati')

    files = list(Path(DATASET_DIR).glob("*.txt"))
    authors = {file.stem: [] for file in files}
    cleaned_all_data = []
    print(authors)

    for file in files:
        with open(file, "r") as f:
            cleaned_text = clean_text(mecab, f.read())
            authors[file.stem] = cleaned_text
            cleaned_all_data += cleaned_text
            print(f"{file.stem}: {cleaned_text[:1]}")

    make_stopdic(cleaned_all_data)
    make_corpus_file(cleaned_all_data, authors)


def clean_text(mecab: MeCab.Tagger, raw_text: str) -> List[str]:
    """
    ルビや入力者注を削除して一文毎に分割,分かち書き
    """
    text = raw_text.replace('\n', '').replace('\u3000', '') # 改行, 全角スペース削除
    text = re.sub('[［《]', '／', text) # 開始括弧を全て／に
    text = re.sub('[］》]', '＼', text) # 終了括弧を全て＼に
    text = re.sub('／[^＼]*?＼', '', text) # 括弧とその間の文字を削除
    text = text.replace('。', '。\n') # 。に改行追加
    text = text.replace('「', '') # 「を削除
    text = text.replace('」', '\n').split('\n') # 」を削除して代わりに改行を追加

    return [mecab.parse(sentence).split(' ') for sentence in text if sentence]


def make_stopdic(lines: List[str]):
    """
    空白区切りの文の集まりのテキストのリストからストップワードの辞書を作成する。
    """
    # AIMathBookのリポジトリでは、make_stopdicの後にclean()していない文章で再度辞書を作成している
    # (02_nlp/cleaning.ipynbのIn [5, 6])
    # おそらくmake_stopdicする方が本来の意図だと思うので、再度辞書は作成していません
    calc_words = Counter()
    for line in lines:
        for word in line:
            calc_words[word] += 1
    sorted_stop = calc_words.most_common()
    print(f"ユニークな単語の数(len(sorted_stop)): {len(sorted_stop)}")

    freq_num = int(len(sorted_stop) * 0.03)
    print(f"ストップワードとして除去する単語の数(freq_num): {freq_num}")

    stop_words = []
    print("High frequency words:")
    for i in range(freq_num + 1):
        stop_words.append(sorted_stop[i][0])
        print(f"{sorted_stop[i][0]}\t{sorted_stop[i][1]}")

    # 作成したストップワードの辞書の保存
    with open(STOPWORD_FILE, 'w') as f:
        f.write('\n'.join(stop_words))


def remove_stopword_bydic(text: List[str]):
    """
    辞書によるストップワードの除去
    """
    # 読み込むストップワード辞書の指定。
    with open(STOPWORD_FILE, "r") as f:
        data = f.read()
        stopwords = data.split('\n')
    lines = []
    for line in text:
        words = []
        for word in line:
            if word not in stopwords:
                words.append(word)
        lines.append(words)
    return lines


def make_corpus_file(cleaned_data: List[List[str]], authors):
    """
    コーパスファイルを作成する
    """
    removed_data = remove_stopword_bydic(cleaned_data)

    unique_words = set()
    for line in removed_data:
        for word in line:
            unique_words.add(word)

    print(f"ストップワードを除外した後の行数：{len(removed_data)}")
    print(f"ユニークな単語の数：{len(unique_words)}")
    with open(UNIQUE_FILE, 'w') as f:
        f.write('\n'.join(list(unique_words)))
    corpus_data = []
    for author, data in authors.items():
        data = remove_stopword_bydic(data)
        for line in data:
            corpus_data.append(author + ',' + ' '.join(line))
    # 作成したコーパスの保存
    with open(CORPUS_FILE, 'w') as f:
        f.write(''.join(corpus_data))


main()

{'akutagawa': [], 'mori': [], 'dazai': []}
akutagawa: [['保吉', 'は', 'ずつ', 'と', '以前', 'から', 'この', '店', 'の', '主人', 'を', '見知', 'つ', 'て', 'ゐる', '。', '\n']]
mori: [['従四', '｜', '位', '下', '左', '近衛', '少将', '兼', '｜', '越中', '守', '細川', '忠利', 'は', '、', '寛永', '十', '八', '年', '｜', '辛', '巳', 'の', '春', '、', 'よそ', 'より', 'は', '早く', '咲く', '領地', '｜', '肥後', '国', 'の', '花', 'を', '見', 'すて', 'て', '、', '五', '十', '四', '万', '石', 'の', '大名', 'の', '晴れ晴れ', 'し', 'い', '行列', 'に', '前後', 'を', '囲ま', 'せ', '、', '南', 'より', '北', 'へ', '歩み', 'を', '運ぶ', '春', 'とともに', '、', '江戸', 'を', '志し', 'て', '参勤', 'の', '途', 'に', '上ろ', 'う', 'と', 'し', 'て', 'いる', 'うち', '、', 'はから', 'ず', '病', 'に', 'かかっ', 'て', '、', '典', '医', 'の', '方', '剤', 'も', '功', 'を', '奏', 'せ', 'ず', '、', '日', 'に', '増し', '重く', 'なる', 'ばかり', 'な', 'ので', '、', '江戸', 'へ', 'は', '出発', '日', '延べ', 'の', '飛脚', 'が', '立つ', '。', '\n']]
dazai: [['賭', '弓', 'に', '、', 'わな', 'なく', '久し', 'う', 'あり', 'て', '、', 'は', 'づしたる', '矢', 'の', '、', 'もて', '離れ', 'て', 'こと', 'かた', 'へ', '行き', 'たる', '。', '\n']]
ユニークな単語の数(le

## 6-7 単語ベクトルの重み付け

$$
\begin{align*}
TF & =\frac{文書内で単語tが現れる回数}{文書内の単語数}\\
IDF & =-\log_{10}\frac{文書内で単語tを含む文章数}{文書内の文章数} =\log_{10}\frac{文書内の文章数}{文章内で単語tを含む文章数}\\
TF-IDF & =TF\cdot IDF=\frac{文書内で単語tが現れる回数}{文書内の単語数}\log_{10}\frac{文書内の文章数}{文書内で単語tを含む文章数}
\end{align*}
$$

例：
- 1000文からなり、単語数10,000個の文書
- ある単語が文書内で100文、120回出現
$$
\begin{align*}
TF & =\frac{120}{10000}=0.012\\
IDF & =-\log_{10}\frac{1000}{100}=\log_{10}10^1=1\\
\end{align*}
$$

## 6-9 完成したモデルの評価

In [7]:
import csv

import gensim
from gensim import models
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix


def main():
    texts, label_ids, id2label = read_copus_file()

    #　テキストとラベルのデータをtrain, testに分割します
    X_train_texts, X_test_texts, y_train, y_test = train_test_split(texts, label_ids, test_size=0.2, random_state=42)

    # テキストデータから辞書を作成します
    dictionary = gensim.corpora.Dictionary(X_train_texts)

    # train, testのテキストデータから、tfidfで重み付けされた単語文書行列を作成します
    X_train_tfidf, tfidf_model = texts_to_tfidf(dictionary, X_train_texts)
    X_test_tfidf, _ = texts_to_tfidf(dictionary, X_test_texts, tfidf_model)

    # trainデータを用いて分類器を構築します
    clf = LogisticRegression(C=1, penalty='l2')
    clf.fit(X_train_tfidf, y_train)

    # testデータを用いて分類器の精度を評価します
    y_pred = clf.predict(X_test_tfidf)
    target_names = list(id2label.values())

    print(classification_report(y_test, y_pred, target_names=target_names))
    print(confusion_matrix(y_test, y_pred))


def read_copus_file():
    with open(CORPUS_FILE, "r") as f:
        corpus_data = list(csv.reader(f))

    # label: author(akutagawa, mori, dazai)
    # texts: corpusファイル各行の単語のlist
    # label_ids: corpusファイル各行のauthor_id(0, 1, 2)のlist
    # label2id: key: author, value: author_idのdict
    # sum_words: 重複を含む文字のlist
    texts, label_ids = [], []
    label2id = {}
    idx_label, idx_sentence = 0, 1
    sum_words = []

    for index, row in enumerate(corpus_data):
        if index == 0:
            continue
        label = row[idx_label]
        if label not in label2id:
            label2id[label] = len(label2id)
        label_ids.append(label2id[label])
        word_list = row[idx_sentence].split(" ")
        texts.append(word_list)

        for line in word_list:
            for word in line:
                sum_words.append(word)

    id2label = {v: k for k, v in label2id.items()}

    return texts, label_ids, id2label


def texts_to_tfidf(dictionary, texts, model = None):
    """テキストデータから、tfidfで重み付けされた単語文書行列を作成する"""
    # 辞書を用いてBoW形式に文章を行列化します
    corpus = [dictionary.doc2bow(text) for text in texts]

    # BoW形式で作成したcorpusをtfidfを用いて重み付けします
    if model is None:
        model = models.TfidfModel(corpus)
    tfidf_corpus = model[corpus]

    num_words = len(dictionary)
    tfidf = gensim.matutils.corpus2dense(tfidf_corpus, num_terms=num_words).T
    return tfidf, model

main()

              precision    recall  f1-score   support

   akutagawa       0.85      0.70      0.77       532
        mori       0.89      0.67      0.76       485
       dazai       0.73      0.94      0.82       758

    accuracy                           0.79      1775
   macro avg       0.82      0.77      0.78      1775
weighted avg       0.81      0.79      0.79      1775

[[372  23 137]
 [ 33 325 127]
 [ 31  18 709]]
